<a href="https://colab.research.google.com/github/Jeffresh/AAED/blob/master/chapter17/Chapter17-gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Networks (Gans) for Synthesizing New Data 

- Introducing generative models for synthesizing new data
- Autoencoders, variational autoencoders (VAEs) , and thei relationships to GANs
- Understanding the building bloks og GANs
- Implementing a simple GAN model to gnerate handwritten digits
- Understanding transposed convolution and *batch normalization (BATchNorm or BN)
- Improving GANs: deep convolutional GANs and GANs using the Wasserstein distance



### Install Tensorflow 2.0 with  gpu support

In [ ]:
! pip install -q tensorflow-gpu>=2.1.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [ ]:
print('GPU Available: ', tf.test.is_gpu_available())

GPU Available:  True


In [ ]:
if tf.test.is_gpu_available():
  device_name = tf.test.gpu_device_name()
else:
  device_name = '/CPU: 0'

print(device_name)

/device:GPU:0


## Imports

In [ ]:
import tensorflow_datasets as tfds
import numpy as np 
import matplotlib.pyplot as plt

## Implementing the generator and the discriminator networks

### Helper functions

In [ ]:
## define a function for the generator:
def make_generator_network(num_hidden_layers=1, num_hidden_units=100, num_output_units=784):

  model = tf.keras.Sequential()

  for i in range(num_hidden_layers):
    model.add(
        tf.keras.layers.Dense(
            units=num_hidden_units, use_bias=False))
    
    model.add(tf.keras.layers.LeakyReLU())
  
  model.add(
      tf.keras.layers.Dense(
          units=num_output_units, activation='tanh'
      )
  )

  return model

In [ ]:
## Define a function fo r the discriminator:

def make_discriminator_network(num_hidden_layers=1, num_hidden_units=100, num_output_units=1):
  
  model = tf.keras.Sequential()
  
  for i in range(num_hidden_layers):
    model.add(
        tf.keras.layers.Dense(units=num_hidden_units)
    )
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(rate=0.5))
  
  model.add(
      tf.keras.layers.Dense(
          units=num_output_units, activation=None
      )
  )
  return model

In [ ]:
image_size = (28, 28)
z_size = 20
mode_z = 'uniform' # 'uniform' vs. 'normal'
gen_hidden_layers = 1
gen_hidden_size= 100
disc_hidden_layers = 1
disc_hidden_size = 100

In [35]:
tf.random.set_seed(1)

gen_model = make_generator_network(
    num_hidden_layers=gen_hidden_layers,
    num_hidden_units=gen_hidden_size,
    num_output_units=np.prod(image_size)
)

In [36]:
gen_model.build(input_shape=(None, z_size))
gen_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              multiple                  2000      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    multiple                  0         
_________________________________________________________________
dense_10 (Dense)             multiple                  79184     
Total params: 81,184
Trainable params: 81,184
Non-trainable params: 0
_________________________________________________________________


In [ ]:
disc_model = make_discriminator_network(num_hidden_layers=disc_hidden_layers, num_hidden_units=disc_hidden_size)
disc_model.build(input_shape=(None, np.prod(image_size)))

In [33]:
disc_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              multiple                  78500     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                  101       
Total params: 78,601
Trainable params: 78,601
Non-trainable params: 0
_________________________________________________________________
